这个程序的作用是根据师兄的c++代码中的公式，自动解码并产生适用于Calculator.AdaptiveQuickSum的代码形式。
下面是示例：

In [1]:
class Token():
    def __init__(self, value, kind):
        if kind not in ['Term', 'Power', 'Start', 'End', 'Coe', 'Add', 'Sub', 'Mul']:
            raise Exception('Invalid Token.')
        self.value = value
        self.kind = kind
        return
    
    def __str__(self):
        return '{} Token, value = {}'.format(self.kind, self.value)

In [2]:
class FormulaParser():
    def __init__(self):
        self.text = ''
        self.output = '('
        self.textlength = 0
        self.cursor = 0
        self.tokens = []
        self.tokens_openBrac = []
        return
    
    #===========================read the formula (character stream)
    def ReadFormula(self, text):
        r'''
        This method will load a character stream from a string (text).
        '''
        assert(type(text) is str)
        self.text = text.replace(' ', '')
        self.text = self.text.replace('pow', 'p')
        self.textlength = len(self.text)
        self.cursor = 0
        return    
    
    
    #===========================tokenize and methods needed, translate character stream into token stream 
    def Tokenize(self):
        r'''
        This method will translate the character stream (self.text) into token stream (self.tokens).
        '''
        while(self.cursor < self.textlength):
            c = self.PeekChar()
            if c == '(':
                self.tokens.append(self.GetStartToken())
            elif c == ')':
                self.tokens.append(self.GetEndToken())
            elif c == 'v':
                self.tokens.append(self.GetTermToken())
            elif c == 'p':
                self.tokens.append(self.GetPowerToken())
            elif c in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                self.tokens.append(self.GetCoeToken())
            elif c == '+':
                self.tokens.append(self.GetAddToken())
            elif c == '-':
                self.tokens.append(self.GetSubToken())
            elif c == '*':
                self.tokens.append(self.GetMulToken())
            else:
                return
        
    def GetChar(self):
        r'''
        Return current character and move on the cursor.
        '''
        c = self.text[self.cursor]
        self.cursor += 1
        return c
    
    def PeekChar(self):
        r'''
        Check current character.
        '''
        if self.cursor < self.textlength:
            return self.text[self.cursor]
        else:
            return False
    
    def GetTermToken(self):
        c = self.GetChar()
        term = ''
        while(True):
            c = self.GetChar()
            if c == '[':
                continue
            elif c == ']':
                return Token(term, 'Term')
            elif c in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                term += c
            else:
                raise Exception('GetTermToken met an error: invalid char "{}".'.format(c))    
    
    def GetPowerToken(self):
        c = self.GetChar()
        power = ''
        while(True):
            c = self.GetChar()
            if c == '(':
                continue
            elif c == 'v':
                term = self.GetTermToken()
            elif c == ',':
                continue
            elif c in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                if self.PeekChar() == ')':
                    power += c
                    power = int(power)
                    self.GetChar()
                    return Token((term.value, power), 'Power')
                else:
                    power += c
            else:
                raise Exception('GetPowerToken met an error: invalid char "{}".'.format(c))
                
    def GetStartToken(self):
        c = self.GetChar()
        if c == '(':
            return Token('', 'Start')
        else:
            raise Exception('GetStartToken met an error: invalid char "{}".'.format(c))
            
    def GetEndToken(self):
        c = self.GetChar()
        if c == ')':
            return Token('', 'End')
        else:
            raise Exception('GetEndToken met an error: invalid char "{}".'.format(c))
            
    def GetCoeToken(self):
        coe = ''
        while(True):
            c = self.PeekChar()
            if c in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                c = self.GetChar()
                coe += c
            elif c == '*':
                coe = int(coe)
                return Token(coe, 'Coe')
                     
    def GetAddToken(self):
        c = self.GetChar()
        if c == '+':
            return Token('', 'Add')
        else:
            raise Exception('GetAddToken met an error: invalid char "{}".'.format(c))

    def GetSubToken(self):
        c = self.GetChar()
        if c == '-':
            return Token('', 'Sub')
        else:
            raise Exception('GetSubToken met an error: invalid char "{}".'.format(c))
            
    def GetMulToken(self):
        c = self.GetChar()
        if c == '*':
            return Token('', 'Mul')
        else:
            raise Exception('GetMulToken met an error: invalid char "{}".'.format(c))

    #===========================open the bracket and derive such form: Coe*Power*Power*... + Coe*Power*... + ... 
    def OpenBracket(self):
        r'''
        Multiple the coefficient into the bracket. Return another token stream.
        After this, the new token stream will only have 'Coe', 'Add' and 'Power' tokens.
        '''
        currentCoe = 1
        innerCoe = 1
        inBracket = False
        isolateCoe = True
        tokens_openBrac = []
        self.tokens_openBrac = []
        for item in self.tokens:
            k = item.kind
            if inBracket:
                if k == 'Sub':
                    innerCoe = -1
                    tokens_openBrac.append(Token('', 'Add'))
                    isolateCoe = True
                elif k == 'Add':
                    innerCoe = 1
                    tokens_openBrac.append(Token('', 'Add'))
                    isolateCoe = True
                elif k == 'Coe':
                    innerCoe *= item.value
                    isolateCoe = True
                elif k == 'Term':
                    if isolateCoe:
                        #if there are multi-terms with the same coefficient, only one coe. token will be added.
                        tokens_openBrac.append(Token(innerCoe*currentCoe, 'Coe'))
                        isolateCoe = False
                    tokens_openBrac.append(Token((item.value, 1), 'Power'))
                elif k == 'Power':
                    if isolateCoe:
                        tokens_openBrac.append(Token(innerCoe*currentCoe, 'Coe'))
                        isolateCoe = False
                    tokens_openBrac.append(Token(item.value, 'Power'))
                elif k == 'End':
                    inBracket = False
                    isolateCoe = True
                    currentCoe = 1
            else:
                if k == 'Sub':
                    currentCoe *= -1
                    tokens_openBrac.append(Token('', 'Add'))
                elif k == 'Add':
                    currentCoe *= 1
                    tokens_openBrac.append(Token('', 'Add'))                   
                elif k == 'Coe':
                    currentCoe *= item.value
                elif k == 'Start':
                    inBracket = True
                elif k == 'Term':
                    tokens_openBrac.append(Token((item.value, 1), 'Power'))
                elif k == 'Power':
                    tokens_openBrac.append(Token(item.value, 'Power'))
        #delete double Adds
        cur = 'Start'
        for item in tokens_openBrac:
            new = item.kind
            if new == 'Add' and cur == 'Add':
                continue
            else:
                self.tokens_openBrac.append(item)
            cur = item.kind
        return
    
    #===========================translate the new token stream into Calculator.AdaptiveQuickSum (iterm, ..., coe, power, ...)
    def Translate(self):
        r'''
        As the tokens_openBrac show such list [coe, power, power, add, coe, ...].
        Splite by 'Add' tokens, and derive several terms (iterm1, ..., coe, power1, ...).
        Form a long string.
        '''
        tmpList = []
        resultList = []
        for item in self.tokens_openBrac:
            k = item.kind
            if k == 'Add':
                resultList.append(self.FormOneTuple(tmpList))
                tmpList = []
            else:
                tmpList.append(item)
        resultList.append(self.FormOneTuple(tmpList))
        #from the resultList, form the string we need
        for item in resultList:
            self.output += item
            self.output += ', '
        self.output = self.output.rstrip(', ')
        self.output += ')'
        return
            
    def FormOneTuple(self, tmpList):
        r'''
        Using a list of tokens, form a tuple (as a string).
        Note that the list is gotten from the spltings from 'Add' tokens.
        The list has such elements: [coe, power1, power2, ...]
        '''
        text = '('
        coe = tmpList.pop(0).value
        termList = []
        powerList = []
        for item in tmpList:
            tmpTerm, tmpPower = item.value
            #Note that the value[0] is string and value[1] is int.
            termList.append(tmpTerm)
            powerList.append(tmpPower)
        for item in termList:
            text += item
            text += ', '
        text += str(coe)
        text += ', '
        for item in powerList:
            text += str(item)
            text += ', '
        text = text.rstrip(', ')
        text += ')'
        return text
        

In [3]:
fp = FormulaParser()

In [4]:
text = '(v[2520])-31*(v[2519])+6*(-v[2535]*v[2525] + v[2460])+15*(-v[2534]*v[2529] + v[2445])+180*(v[2518])-90*(-v[2535]*v[2524] + v[2459])-15*(-v[2533]*v[2529] + v[2427])+15*(-v[2514]*v[2529] + 2*pow(v[2535], 2)*v[2529] - 2*v[2535]*v[2464] + v[2254])+10*(v[2496] - pow(v[2532], 2))-105*(-v[2534]*v[2528] + v[2444])-60*(-v[2532]*v[2531] + v[2412])+60*(2*v[2535]*v[2534]*v[2532] - v[2535]*v[2448] - v[2534]*v[2467] - v[2469]*v[2532] + v[1835])-390*(v[2517])+300*(-v[2535]*v[2523] + v[2458])+105*(-v[2533]*v[2528] + v[2426])-105*(-v[2514]*v[2528] + 2*pow(v[2535], 2)*v[2528] - 2*v[2535]*v[2463] + v[2253])+40*(-v[2532]*v[2530] + v[2411])-60*(2*v[2535]*v[2533]*v[2532] - v[2535]*v[2430] - v[2533]*v[2467] - v[2468]*v[2532] + v[1817])+20*(6*v[2514]*v[2535]*v[2532] - 3*v[2514]*v[2467] - v[2513]*v[2532] - 6*pow(v[2535], 3)*v[2532] + 6*pow(v[2535], 2)*v[2467] - 3*v[2535]*v[2257] + v[2237])+15*(-3*v[2506]*v[2534] + v[2505] + 2*pow(v[2534], 3))+180*(-v[2534]*v[2527] + v[2443])+90*(v[2493] - pow(v[2531], 2))-180*(2*v[2535]*v[2534]*v[2531] - v[2535]*v[2447] - v[2534]*v[2466] - v[2469]*v[2531] + v[1834])-45*(-v[2506]*v[2533] + 2*pow(v[2534], 2)*v[2533] - 2*v[2534]*v[2449] + v[2108])+45*(-v[2514]*v[2506] + 2*v[2514]*pow(v[2534], 2) + 2*pow(v[2535], 2)*v[2506] - 6*pow(v[2535], 2)*pow(v[2534], 2) - 2*v[2535]*v[2131] + 8*v[2535]*v[2534]*v[2469] + v[1935] - 2*v[2534]*v[2259] - 2*pow(v[2469], 2))+360*(v[2516])-360*(-v[2535]*v[2522] + v[2457])-180*(-v[2533]*v[2527] + v[2425])+180*(-v[2514]*v[2527] + 2*pow(v[2535], 2)*v[2527] - 2*v[2535]*v[2462] + v[2252])-120*(-v[2531]*v[2530] + v[2395])+180*(2*v[2535]*v[2533]*v[2531] - v[2535]*v[2429] - v[2533]*v[2466] - v[2468]*v[2531] + v[1816])-60*(6*v[2514]*v[2535]*v[2531] - 3*v[2514]*v[2466] - v[2513]*v[2531] - 6*pow(v[2535], 3)*v[2531] + 6*pow(v[2535], 2)*v[2466] - 3*v[2535]*v[2256] + v[2236])-90*(-v[2534]*v[2526] + v[2442])+120*(2*v[2535]*v[2534]*v[2530] - v[2535]*v[2446] - v[2534]*v[2465] - v[2469]*v[2530] + v[1833])+45*(-v[2534]*v[2501] + 2*v[2534]*pow(v[2533], 2) + v[2061] - 2*v[2533]*v[2449])-90*(2*v[2514]*v[2534]*v[2533] - v[2514]*v[2449] - 6*pow(v[2535], 2)*v[2534]*v[2533] + 2*pow(v[2535], 2)*v[2449] + 4*v[2535]*v[2534]*v[2468] + 4*v[2535]*v[2469]*v[2533] - 2*v[2535]*v[1836] - v[2534]*v[2258] - v[2259]*v[2533] - 2*v[2469]*v[2468] + v[1401])+15*(6*pow(v[2514], 2)*v[2534] - 36*v[2514]*pow(v[2535], 2)*v[2534] + 24*v[2514]*v[2535]*v[2469] - 6*v[2514]*v[2259] + 8*v[2513]*v[2535]*v[2534] - 4*v[2513]*v[2469] - v[2512]*v[2534] + 24*pow(v[2535], 4)*v[2534] - 24*pow(v[2535], 3)*v[2469] + 12*pow(v[2535], 2)*v[2259] - 4*v[2535]*v[2239] + v[2219])-120*(v[2515])+144*(-v[2535]*v[2521] + v[2456])+90*(-v[2533]*v[2526] + v[2424])-90*(-v[2514]*v[2526] + 2*pow(v[2535], 2)*v[2526] - 2*v[2535]*v[2461] + v[2251])+40*(v[2490] - pow(v[2530], 2))-120*(2*v[2535]*v[2533]*v[2530] - v[2535]*v[2428] - v[2533]*v[2465] - v[2468]*v[2530] + v[1815])+40*(6*v[2514]*v[2535]*v[2530] - 3*v[2514]*v[2465] - v[2513]*v[2530] - 6*pow(v[2535], 3)*v[2530] + 6*pow(v[2535], 2)*v[2465] - 3*v[2535]*v[2255] + v[2235])-15*(-3*v[2501]*v[2533] + v[2500] + 2*pow(v[2533], 3))+45*(-v[2514]*v[2501] + 2*v[2514]*pow(v[2533], 2) + 2*pow(v[2535], 2)*v[2501] - 6*pow(v[2535], 2)*pow(v[2533], 2) - 2*v[2535]*v[2127] + 8*v[2535]*v[2533]*v[2468] + v[1931] - 2*v[2533]*v[2258] - 2*pow(v[2468], 2))-15*(6*pow(v[2514], 2)*v[2533] - 36*v[2514]*pow(v[2535], 2)*v[2533] + 24*v[2514]*v[2535]*v[2468] - 6*v[2514]*v[2258] + 8*v[2513]*v[2535]*v[2533] - 4*v[2513]*v[2468] - v[2512]*v[2533] + 24*pow(v[2535], 4)*v[2533] - 24*pow(v[2535], 3)*v[2468] + 12*pow(v[2535], 2)*v[2258] - 4*v[2535]*v[2238] + v[2218])+1*(30*pow(v[2514], 3) - 270*pow(v[2514], 2)*pow(v[2535], 2) + 120*v[2514]*v[2513]*v[2535] - 15*v[2514]*v[2512] + 360*v[2514]*pow(v[2535], 4) - 10*pow(v[2513], 2) - 120*v[2513]*pow(v[2535], 3) + 30*v[2512]*pow(v[2535], 2) - 6*v[2511]*v[2535] + v[2510] - 120*pow(v[2535], 6))'

In [5]:
fp.ReadFormula(text)

In [6]:
fp.Tokenize()

In [7]:
fp.OpenBracket()

In [8]:
# for item in fp.tokens:
#     print(item.kind, end= ' ')

In [9]:
fp.Translate()

In [10]:
fp.output

'((2520, 1, 1), (2519, -31, 1), (2535, 2525, -6, 1, 1), (2460, 6, 1), (2534, 2529, -15, 1, 1), (2445, 15, 1), (2518, 180, 1), (2535, 2524, 90, 1, 1), (2459, -90, 1), (2533, 2529, 15, 1, 1), (2427, -15, 1), (2514, 2529, -15, 1, 1), (2535, 2529, 30, 2, 1), (2535, 2464, -30, 1, 1), (2254, 15, 1), (2496, 10, 1), (2532, -10, 2), (2534, 2528, 105, 1, 1), (2444, -105, 1), (2532, 2531, 60, 1, 1), (2412, -60, 1), (2535, 2534, 2532, 120, 1, 1, 1), (2535, 2448, -60, 1, 1), (2534, 2467, -60, 1, 1), (2469, 2532, -60, 1, 1), (1835, 60, 1), (2517, -390, 1), (2535, 2523, -300, 1, 1), (2458, 300, 1), (2533, 2528, -105, 1, 1), (2426, 105, 1), (2514, 2528, 105, 1, 1), (2535, 2528, -210, 2, 1), (2535, 2463, 210, 1, 1), (2253, -105, 1), (2532, 2530, -40, 1, 1), (2411, 40, 1), (2535, 2533, 2532, -120, 1, 1, 1), (2535, 2430, 60, 1, 1), (2533, 2467, 60, 1, 1), (2468, 2532, 60, 1, 1), (1817, -60, 1), (2514, 2535, 2532, 120, 1, 1, 1), (2514, 2467, -60, 1, 1), (2513, 2532, -20, 1, 1), (2535, 2532, -120, 3, 1), (